<a href="https://colab.research.google.com/github/viyercal/ai-scientist-safety/blob/Repo-Setup/applications/ai-scientist-v2-safety/src/AI_Scientist_V2_Safety.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# 1. Clone the repo
'''%cd /content/drive/MyDrive/
!git clone https://github.com/SakanaAI/AI-Scientist-v2.git'''
%cd /content/drive/MyDrive/AI-Scientist-v2

/content/drive/MyDrive/AI-Scientist-v2


In [ ]:
'''# 2. System packages for PDF + LaTeX-ish tooling
!apt-get update -y
!apt-get install -y poppler-utils chktex'''


In [ ]:
# 3. Python requirements
!pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.5/371.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 139.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.6 MB/s eta 0:00:00


In [1]:
import os
from google.colab import userdata
os.environ["OPENROUTER_API_KEY"] = userdata.get('open_router')
os.environ["OPENAI_API_KEY"] = os.environ["OPENROUTER_API_KEY"]
os.environ["OPENAI_BASE_URL"] = "https://openrouter.ai/api/v1"

In [ ]:
#!mkdir -p ai_scientist/safety

#create config file (code hidden)
#create safety config yaml with potentially dangerous imported packages
#create static linter via ast to test imports, functions, etc.
#create a sandboxed runner env that works with colab
#create a guardian object that will wrap around the ai scientist calls w/safety logic
#create a guardian runner for simplicity of access

In [ ]:
!python launch_scientist_bfts.py \
 --load_ideas "ai_scientist/ideas/curriculum_learning.json" \
 --add_dataset_ref \
 --model_writeup gpt-4o \
 --model_citation gpt-4o \
 --model_review gpt-4o \
 --model_agg_plots o1 \
 --num_cite_rounds 5

Streaming output truncated to the last 5000 lines.
more pronounced decrease in both training and validation loss, with the 
validation loss closely following the training loss. This suggests that the 
model is learning effectively without significant overfitting, indicating that 
this learning rate may be more appropriate for the task.","analysis":"The loss 
curves for the learning rate of 0.01 exhibit a rapid decrease in training loss, 
but the validation loss stagnates at a low value after a sharp drop, indicating 
that the model may be overfitting to the training data. The gap between training
and validation loss suggests that while the model performs well on training 
data, it may not generalize effectively to new 
data."}],"valid_plots_received":true,"vlm_feedback_summary":"The analysis of the
loss curves indicates that different learning rates significantly impact model 
performance. A learning rate of 0.001 appears optimal, balancing effective 
training and generalization, while

In [ ]:
from ai_scientist.treesearch.interpreter import Interpreter

interp = Interpreter(working_dir=".", timeout=5)

print("=== UNSAFE ===")
r1 = interp.run("import subprocess\nimport os\nos.system('echo hi')")
print("\n".join(r1.term_out))

print("\n=== SAFE ===")
r2 = interp.run("x = 1 + 2\nprint('x is', x)")
print("\n".join(r2.term_out))


In [ ]:
'''%%writefile ai_scientist/safety/sandbox_runner.py
import json
import subprocess
import time
from pathlib import Path
from typing import Optional
from .config import SafetyConfig, SafetyReport, SafetyIssue


def run_in_sandbox(script_path: str | Path, cfg: SafetyConfig) -> SafetyReport:
    script_path = Path(script_path).resolve()

    # Sandbox dir: sibling "sandbox" under the experiment dir
    sandbox_dir = script_path.parent / "sandbox"
    sandbox_dir.mkdir(exist_ok=True)

    start = time.time()
    try:
        # IMPORTANT CHANGE:
        # Use the absolute script_path, not just script_path.name,
        # while cwd is sandbox_dir.
        proc = subprocess.run(
            ["python", str(script_path)],
            cwd=str(sandbox_dir),
            capture_output=True,
            text=True,
            timeout=cfg.max_runtime_sec,
        )
        runtime = time.time() - start
    except subprocess.TimeoutExpired:
        issue = SafetyIssue(
            severity="error",
            code="RUNTIME_TIMEOUT",
            detail=f"Script exceeded max_runtime_sec={cfg.max_runtime_sec}",
            location=None,
        )
        return SafetyReport(passed=False, issues=[issue], risk_score=1.0)

    issues = []
    if proc.returncode != 0:
        issues.append(
            SafetyIssue(
                severity="warning",
                code="NONZERO_EXIT",
                detail=f"Script exited with code {proc.returncode}. stderr: {proc.stderr[:500]}",
                location=None,
            )
        )

    # Log outputs inside sandbox
    (sandbox_dir / "stdout.txt").write_text(proc.stdout or "")
    (sandbox_dir / "stderr.txt").write_text(proc.stderr or "")

    risk_score = 0.0 if not issues else 0.3
    return SafetyReport(passed=True, issues=issues, risk_score=risk_score)
'''

Writing ai_scientist/safety/sandbox_runner.py


In [ ]:
'''%%writefile test_safe.py
import math

def main():
    xs = [0.1 * i for i in range(10)]
    ys = [math.sin(x) for x in xs]
    print("mean:", sum(ys) / len(ys))

if __name__ == "__main__":
    main()
'''

Writing test_safe.py


In [ ]:
'''%%writefile test_unsafe.py
import os

os.system("rm -rf /")  # lol nope
'''

Writing test_unsafe.py


In [ ]:
'''%%writefile ai_scientist/safety/guardian.py
from __future__ import annotations
import json
from pathlib import Path
from .config import SafetyConfig, SafetyReport, SafetyIssue, load_safety_config
from .static_analyzer import analyze_file
from .sandbox_runner import run_in_sandbox


def run_with_guardian(script_path: str | Path, cfg: SafetyConfig) -> SafetyReport:
    script_path = Path(script_path)
    static_report = analyze_file(script_path, cfg)

    if not static_report.passed:
        # Just return static report; do not execute
        return static_report

    runtime_report = run_in_sandbox(script_path, cfg)

    # Merge reports
    issues = list(static_report.issues) + list(runtime_report.issues)
    risk_score = max(static_report.risk_score, runtime_report.risk_score)
    passed = static_report.passed and runtime_report.passed
    return SafetyReport(passed=passed, issues=issues, risk_score=risk_score)


def report_to_dict(report: SafetyReport) -> dict:
    return {
        "passed": report.passed,
        "risk_score": report.risk_score,
        "issues": [
            {
                "severity": i.severity,
                "code": i.code,
                "detail": i.detail,
                "location": i.location,
            }
            for i in report.issues
        ],
    }


def main_cli(script_path: str, cfg_path: str = "ai_scientist/safety/safety_config.yaml"):
    cfg = load_safety_config(cfg_path)
    report = run_with_guardian(script_path, cfg)
    print(json.dumps(report_to_dict(report), indent=2))


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument("--script", required=True, help="Path to experiment script")
    parser.add_argument(
        "--config",
        default="ai_scientist/safety/safety_config.yaml",
        help="Path to safety config YAML",
    )
    args = parser.parse_args()
    main_cli(args.script, args.config)
'''

Writing ai_scientist/safety/guardian.py


In [ ]:
'''%%writefile ai_scientist/safety/static_analyzer.py
import ast
from pathlib import Path
from typing import List
from .config import SafetyConfig, SafetyReport, SafetyIssue


class _Visitor(ast.NodeVisitor):
    def __init__(self, cfg: SafetyConfig):
        self.cfg = cfg
        self.issues: List[SafetyIssue] = []

    def visit_Import(self, node: ast.Import) -> None:
        for alias in node.names:
            name = alias.name.split(".")[0]
            if name in self.cfg.blocked_modules:
                self.issues.append(
                    SafetyIssue(
                        severity="error",
                        code="BLOCKED_IMPORT",
                        detail=f"Importing blocked module '{name}'",
                        location=f"line {node.lineno}",
                    )
                )
        self.generic_visit(node)

    def visit_ImportFrom(self, node: ast.ImportFrom) -> None:
        if node.module:
            name = node.module.split(".")[0]
            if name in self.cfg.blocked_modules:
                self.issues.append(
                    SafetyIssue(
                        severity="error",
                        code="BLOCKED_IMPORT",
                        detail=f"Importing from blocked module '{name}'",
                        location=f"line {node.lineno}",
                    )
                )
        self.generic_visit(node)

    def visit_Call(self, node: ast.Call) -> None:
        # Function name: might be Name or Attribute
        func_name = None
        if isinstance(node.func, ast.Name):
            func_name = node.func.id
        elif isinstance(node.func, ast.Attribute):
            # something.func; take "module.func"
            base = []
            cur = node.func
            while isinstance(cur, ast.Attribute):
                base.append(cur.attr)
                cur = cur.value
            if isinstance(cur, ast.Name):
                base.append(cur.id)
            base_rev = list(reversed(base))
            func_name = ".".join(base_rev)

        if func_name:
            # Check blocked functions and common dangerous patterns
            for blocked in self.cfg.blocked_functions:
                if func_name == blocked or func_name.endswith("." + blocked):
                    self.issues.append(
                        SafetyIssue(
                            severity="error",
                            code="BLOCKED_CALL",
                            detail=f"Call to blocked function '{func_name}'",
                            location=f"line {node.lineno}",
                        )
                    )
            if func_name.startswith("os.system") or func_name.startswith("subprocess."):
                self.issues.append(
                    SafetyIssue(
                        severity="error",
                        code="DANGEROUS_PROC",
                        detail=f"Spawning subprocess via '{func_name}'",
                        location=f"line {node.lineno}",
                    )
                )

        self.generic_visit(node)


def analyze_code_str(code: str, cfg: SafetyConfig) -> SafetyReport:
    try:
        tree = ast.parse(code)
    except SyntaxError as e:
        issue = SafetyIssue(
            severity="error",
            code="SYNTAX_ERROR",
            detail=str(e),
            location=f"line {e.lineno}",
        )
        return SafetyReport(passed=False, issues=[issue], risk_score=1.0)

    visitor = _Visitor(cfg)
    visitor.visit(tree)

    has_error = any(i.severity == "error" for i in visitor.issues)
    risk_score = 1.0 if has_error else (0.2 if visitor.issues else 0.0)
    return SafetyReport(passed=not has_error, issues=visitor.issues, risk_score=risk_score)


def analyze_file(path: str | Path, cfg: SafetyConfig) -> SafetyReport:
    path = Path(path)
    code = path.read_text()
    return analyze_code_str(code, cfg)
'''

Writing ai_scientist/safety/static_analyzer.py


In [ ]:
'''%%writefile ai_scientist/safety/safety_config.yaml
blocked_modules:
  - os
  - subprocess
  - socket
  - shutil
  - sys
  - requests
  - httpx
  - urllib
  - urllib3
  - aiohttp
  - websocket
  - paramiko

blocked_functions:
  - eval
  - exec
  - compile
  - open

allow_network: false
max_runtime_sec: 600
max_ram_mb: 4096
allow_fs_outside_sandbox: false
'''

Writing ai_scientist/safety/safety_config.yaml


In [ ]:
'''import yaml
from pathlib import Path

cfg_path = Path("/content/drive/MyDrive/AI-Scientist-v2/bfts_config.yaml")

with open(cfg_path, "r") as f:
    cfg = yaml.safe_load(f)

# Use a cheap, OpenAI-style model for all agent-side calls
mini_model = "gpt-4o-mini-2024-07-18"

cfg["agent"]["code"]["model"] = mini_model
cfg["agent"]["feedback"]["model"] = mini_model
cfg["agent"]["vlm_feedback"]["model"] = mini_model

# (Optional but nice) Also make sure the report model is OpenAI-style:
if "report" in cfg and "model" in cfg["report"]:
    cfg["report"]["model"] = "gpt-4o-2024-11-20"

with open(cfg_path, "w") as f:
    yaml.safe_dump(cfg, f, sort_keys=False)

print("Updated bfts_config.yaml LLM models:")
print("  agent.code.model      =", cfg["agent"]["code"]["model"])
print("  agent.feedback.model  =", cfg["agent"]["feedback"]["model"])
print("  agent.vlm_feedback    =", cfg["agent"]["vlm_feedback"]["model"])
print("  report.model          =", cfg.get("report", {}).get("model"))
'''

Updated bfts_config.yaml LLM models:
  agent.code.model      = gpt-4o-mini-2024-07-18
  agent.feedback.model  = gpt-4o-mini-2024-07-18
  agent.vlm_feedback    = gpt-4o-mini-2024-07-18
  report.model          = gpt-4o-2024-11-20


In [ ]:
'''%%writefile ai_scientist/safety/config.py
from __future__ import annotations
from dataclasses import dataclass, field
from typing import List, Literal
import yaml
from pathlib import Path


@dataclass
class SafetyIssue:
    severity: Literal["info", "warning", "error"]
    code: str
    detail: str
    location: str | None = None


@dataclass
class SafetyReport:
    passed: bool
    issues: List[SafetyIssue] = field(default_factory=list)
    risk_score: float = 0.0  # 0–1, higher = riskier


@dataclass
class SafetyConfig:
    allowed_modules: List[str] = field(default_factory=list)
    blocked_modules: List[str] = field(
        default_factory=lambda: [
            "os", "subprocess", "socket", "shutil", "sys",
            "requests", "httpx", "urllib", "urllib3",
            "aiohttp", "websocket", "paramiko"
        ]
    )
    blocked_functions: List[str] = field(
        default_factory=lambda: ["eval", "exec", "compile", "open"]
    )
    allow_network: bool = False
    max_runtime_sec: int = 600
    max_ram_mb: int = 4096  # placeholder; you can implement RLIMIT later
    allow_fs_outside_sandbox: bool = False


def load_safety_config(path: str | Path) -> SafetyConfig:
    path = Path(path)
    if not path.exists():
        # fallback to defaults
        return SafetyConfig()
    cfg_dict = yaml.safe_load(path.read_text())
    return SafetyConfig(**cfg_dict)
'''

Writing ai_scientist/safety/config.py


In [ ]:
!python ai_scientist/perform_ideation_temp_free.py \
 --workshop-file "ai_scientist/ideas/curriculum_learning.md" \
 --model gpt-4o \
 --max-num-generations 20 \
 --num-reflections 5

In [ ]:
!python -m ai_scientist.safety.guardian --script test_safe.py


{
  "passed": true,
  "risk_score": 0.0,
  "issues": []
}


In [ ]:
!python -m ai_scientist.safety.guardian --script test_unsafe.py


{
  "passed": true,
  "risk_score": 0.0,
  "issues": []
}


In [ ]:
%%bash
cd /content/drive/MyDrive/AI-Scientist-v2

python launch_scientist_bfts.py \
  --load_ideas "ai_scientist/ideas/i_cant_believe_its_not_better.json" \
  --idea_idx 0 \
  --load_code \
  --add_dataset_ref \
  --skip_writeup \
  --skip_review \
  --num_cite_rounds 0



Process is terminated.
